In [1]:
# from transformers import SegformerForSemanticSegmentation
import torch
from torch import nn
import numpy as np


In [5]:
temp = torch.load("./helper/models/saved/Unet/Unet-20250325-201910.ckpt")
type(temp)

dict

In [6]:
temp

{'epoch': 0,
 'global_step': 24,
 'pytorch-lightning_version': '2.5.1',
 'state_dict': OrderedDict([('model.encoder.conv1.weight',
               tensor([[[[ 8.1118e-03, -1.5897e-02, -4.6560e-03,  ...,  3.0839e-02,
                          -4.7166e-02,  2.6874e-02],
                         [ 9.7651e-03, -1.5023e-03, -4.6663e-03,  ...,  4.5993e-02,
                           4.6260e-02,  2.3671e-02],
                         [-1.7627e-02, -4.4185e-02,  2.0438e-02,  ..., -2.4046e-02,
                           1.9426e-02,  2.1831e-02],
                         ...,
                         [ 1.7331e-02,  3.6168e-02, -2.0543e-04,  ..., -2.8026e-02,
                          -3.3497e-02,  1.6740e-02],
                         [-2.5342e-02, -2.9636e-02,  1.1428e-02,  ...,  5.7268e-04,
                          -5.7840e-03,  1.4694e-02],
                         [-7.6857e-03, -5.5187e-02, -9.9899e-03,  ...,  4.3216e-03,
                           6.0979e-03,  3.3930e-02]],
               


In [7]:
from helper.models.unet import *
model = UNet.load_from_checkpoint('./helper/models/saved/Unet/Unet-20250325-201910.ckpt')

g:\Universe\Materials\Github Repos\Drainage-detection-topography\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initialized Unet

Loaded {'uid': '20250325-205202', 'num_epochs': 100, 'batch_size': 64, 'opt': 'str', 'crit': 'CrossEntropy', 'learning_rate': 0.0001, 'scheduler': None, 'test_iter': 0}
 Loaded 20250325-205202
Unet initialized with hyperparams: {'uid': '20250325-205202', 'num_epochs': 100, 'batch_size': 64, 'opt': 'str', 'crit': 'CrossEntropy', 'learning_rate': 0.0001, 'scheduler': None, 'test_iter': 0}


In [13]:
print(model.hyperparameters)

AttributeError: 'UNet' object has no attribute 'hyperparameters'

In [36]:
print(model.segformer.encoder.patch_embeddings[0].proj)

Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))


In [64]:
from segmentation_models_pytorch import Unet, UnetPlusPlus
model = UnetPlusPlus(in_channels=4, classes=2)

In [65]:
print(model)

UnetPlusPlus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

In [55]:
summary(model, input_size=(batch_size, 4, 256, 256))

Layer (type:depth-idx)                        Output Shape              Param #
UnetPlusPlus                                  [16, 2, 256, 256]         --
├─ResNetEncoder: 1-1                          [16, 4, 256, 256]         --
│    └─Conv2d: 2-1                            [16, 64, 128, 128]        12,544
│    └─BatchNorm2d: 2-2                       [16, 64, 128, 128]        128
│    └─ReLU: 2-3                              [16, 64, 128, 128]        --
│    └─MaxPool2d: 2-4                         [16, 64, 64, 64]          --
│    └─Sequential: 2-5                        [16, 64, 64, 64]          --
│    │    └─BasicBlock: 3-1                   [16, 64, 64, 64]          73,984
│    │    └─BasicBlock: 3-2                   [16, 64, 64, 64]          73,984
│    │    └─BasicBlock: 3-3                   [16, 64, 64, 64]          73,984
│    └─Sequential: 2-6                        [16, 128, 32, 32]         --
│    │    └─BasicBlock: 3-4                   [16, 128, 32, 32]         230,14

In [62]:
from torchinfo import summary

batch_size = 16
for name in model.encoder.conv1.parameters():
    print(f"{name}-param")
    

for param in model.encoder.conv2d.parameters():
    param.requires_grad = True
    
summary(model, input_size=(batch_size, 4, 256, 256))

Parameter containing:
tensor([[[[ 4.0582e-03, -5.1819e-03,  5.9129e-03,  ...,  3.6804e-02,
            2.2995e-02,  1.9049e-02],
          [ 3.0810e-02,  2.3472e-02,  2.4199e-02,  ...,  2.4859e-02,
            2.2316e-02,  3.1302e-02],
          [ 3.7139e-03, -2.3779e-02, -4.5982e-02,  ..., -7.3120e-02,
           -8.7007e-02, -9.1435e-02],
          ...,
          [-9.2153e-03, -1.8631e-02, -6.9789e-03,  ...,  1.2835e-02,
            1.8473e-03,  1.2545e-02],
          [ 2.9338e-03,  3.3403e-03,  2.7236e-02,  ...,  7.7786e-02,
            5.5480e-02,  4.4314e-02],
          [ 1.2588e-02,  6.6676e-03,  2.3484e-02,  ...,  7.2723e-02,
            6.2812e-02,  7.2727e-02]],

         [[-5.7894e-03, -6.5783e-03,  1.0607e-02,  ...,  2.5425e-02,
            1.9112e-02,  1.8206e-02],
          [ 4.0471e-02,  3.3508e-02,  2.5745e-02,  ...,  1.0044e-02,
            1.4351e-02,  2.8496e-02],
          [ 7.6879e-04, -4.0885e-02, -7.6688e-02,  ..., -1.4423e-01,
           -1.4996e-01, -1.3644e-01]

AttributeError: 'ResNetEncoder' object has no attribute 'conv2d'

In [38]:
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/segformer-b1-finetuned-ade-512-512",
    num_labels=2,
    ignore_mismatched_sizes=True,
    id2label={0: "background", 1: "object"},  # пример
    label2id={"background": 0, "object": 1},
)

old_conv = model.segformer.encoder.patch_embeddings[0].proj

# Создадим новый сверточный слой с in_channels=4, сохраняя остальные параметры
new_conv = nn.Conv2d(
    in_channels=4,
    out_channels=old_conv.out_channels,
    kernel_size=old_conv.kernel_size,
    stride=old_conv.stride,
    padding=old_conv.padding,
    bias=(old_conv.bias is not None)
)

with torch.no_grad():
    # Копируем веса для первых трёх каналов из предобученной модели
    new_conv.weight[:, :3, :, :] = old_conv.weight
    # Инициализируем вес для 4-го канала как среднее по первым трем каналам
    new_conv.weight[:, 3:4, :, :] = old_conv.weight.mean(dim=1, keepdim=True)
    if old_conv.bias is not None:
        new_conv.bias.copy_(old_conv.bias)

# Подменяем старый сверточный слой новым
model.segformer.encoder.patch_embeddings[0].proj = new_conv

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b1-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([2, 256, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
summary(model, (16, 4, 256, 256))

Layer (type:depth-idx)                                                      Output Shape              Param #
SegformerForSemanticSegmentation                                            [16, 2, 64, 64]           --
├─SegformerModel: 1-1                                                       [16, 64, 64, 64]          --
│    └─SegformerEncoder: 2-1                                                [16, 64, 64, 64]          --
│    │    └─ModuleList: 3-10                                                --                        (recursive)
│    │    └─ModuleList: 3-11                                                --                        (recursive)
│    │    └─ModuleList: 3-12                                                --                        (recursive)
│    │    └─ModuleList: 3-10                                                --                        (recursive)
│    │    └─ModuleList: 3-11                                                --                        (recursive)
│    

In [6]:
from torchsummary import summary

In [16]:
model = model.to('cpu')

In [25]:
print(model)

SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [24]:
from torchinfo import summary

model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b1-finetuned-ade-512-512",
    num_labels=2,
    ignore_mismatched_sizes=True,
    id2label={0: "background", 1: "object"},  # пример
    label2id={"background": 0, "object": 1},)
batch_size = 16
summary(model, input_size=(batch_size, 3, 256, 256))

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b1-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 256, 1, 1]) in the checkpoint and torch.Size([2, 256, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Layer (type:depth-idx)                                                      Output Shape              Param #
SegformerForSemanticSegmentation                                            [16, 2, 64, 64]           --
├─SegformerModel: 1-1                                                       [16, 64, 64, 64]          --
│    └─SegformerEncoder: 2-1                                                [16, 64, 64, 64]          --
│    │    └─ModuleList: 3-10                                                --                        (recursive)
│    │    └─ModuleList: 3-11                                                --                        (recursive)
│    │    └─ModuleList: 3-12                                                --                        (recursive)
│    │    └─ModuleList: 3-10                                                --                        (recursive)
│    │    └─ModuleList: 3-11                                                --                        (recursive)
│    